# Train a Convolutional Neural Network (CNN) on EEG data

In [12]:
from bundle.DataCraft import * 
from bundle.ModelCraft import * 
import numpy as np

data = load_sentence_eeg_prob_data()

if data:
    # Print example of converted item
    print(f"Loaded {len(data)} items.")
    
    # Convert to NumPy array to inspect the full shape
    eeg_array = np.array(data[0]["eeg_chunk"])
    print("Shape of EEG chunk:", eeg_array.shape)

    if data:
        print("Example of first item:", data[1]["character"])
        print("Example of first item:", data[1]["eeg_chunk"][0])

Attempting to load processed data from: ../../data/sentences_eeg.pkl
Successfully loaded processed data.
Loaded 7868 items.
Shape of EEG chunk: (31, 78, 64)
Example of first item: H
Example of first item: [[ 1.9504154   2.2442322   1.9698964  ...  1.4474702   1.3025287
   1.7617115 ]
 [ 1.8555712   2.1287162   1.8950977  ...  1.6038033   1.4954389
   1.889984  ]
 [ 1.7028829   1.9374998   1.7584169  ...  1.7040414   1.6331682
   1.9601816 ]
 ...
 [ 0.40570083 -0.01885907  0.21926583 ... -0.7260041  -0.72525847
  -0.72926086]
 [ 0.45789889  0.0929183   0.399929   ... -0.6825177  -0.64262927
  -0.7352173 ]
 [ 0.46558765  0.17591506  0.5342773  ... -0.6526491  -0.5673434
  -0.75709546]]


In [13]:
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


# Prepare data
X = np.array([d["eeg_chunk"] for d in data])
y = np.array([d["character"] for d in data])

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_encoded)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Reshape X for CNN (add channel dimension)
X_train = X_train.reshape(X_train.shape[0], 31, 78, 64, 1)
X_test = X_test.reshape(X_test.shape[0], 31, 78, 64, 1)

# Create and compile model
model = create_eeg_cnn_model(input_shape=(31, 78, 64, 1), num_classes=len(label_encoder.classes_))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Data preprocessing and model compilation complete.")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("Number of classes:", len(label_encoder.classes_))

# Save preprocessed data and model for later use
with open("preprocessed_data.pkl", "wb") as f:
    pickle.dump({
        "X_train": X_train,
        "X_test": X_test,
        "y_train": y_train,
        "y_test": y_test,
        "label_encoder": label_encoder
    }, f)

model.save("initial_eeg_cnn_model.h5")


D:\University\Year 4 semester 2\ENCS5300\BrainWaveResearch\ecd\venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Data preprocessing and model compilation complete.
X_train shape: (6294, 31, 78, 64, 1)
y_train shape: (6294, 28)
Number of classes: 28


In [14]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
import pickle
from tensorflow.keras.models import load_model

# Load preprocessed data
with open("preprocessed_data.pkl", "rb") as f:
    data = pickle.load(f)

X_train = data["X_train"]
X_test = data["X_test"]
y_train = data["y_train"]
y_test = data["y_test"]
num_classes = len(data["label_encoder"].classes_)

# Create and compile model (instead of loading a pre-compiled one)
model = create_eeg_cnn_model(input_shape=(31, 78, 64, 1), num_classes=num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Save the trained model
model.save("trained_eeg_cnn_model.h5")

print("Model training and evaluation complete.")

D:\University\Year 4 semester 2\ENCS5300\BrainWaveResearch\ecd\venv\lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/10
  5/197 ━━━━━━━━━━━━━━━━━━━━ 18:01 6s/step - accuracy: 0.0589 - loss: 4.4036


KeyboardInterrupt

